In [ ]:
import pandas as pd
from time import sleep
import re
import threading
import random
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import os
from itertools import product

dpath = '首经贸/应用经济学/'
with open(dpath+'succeed.txt','r',encoding='utf-8') as f:
    data = f.readlines()
succeed = [x.strip() for x in data if x != ''] #记录完成的名单
url = 'http://find.nlc.cn/show/advancedSearch'
options = webdriver.ChromeOptions()
user_agent_list = [ \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60" \
        "Opera/8.0 (Windows NT 5.1; U; en)"\
        "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0" \
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER" \
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0" \
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)" \
    ]

index_path = 'D:\\codes\\GitProject\\工作区\\首经贸数据采集项目\\专著数据\\首经贸\\应用经济学'
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': index_path}  #下载路径
options.add_experimental_option('prefs', prefs)

#def get_info(scholars):
# global options
# global succeed

def get_info(scholars):
    global succeed
    global options
    my_counts = 0
    random.shuffle(scholars)
    for scholar in scholars:
        if scholar in succeed:
            continue
        my_counts+=1
        if my_counts%3 == 0: #及时保存succeed
            succeed = list(set(succeed))
            with open(dpath+'succeed.txt','w',encoding='utf-8') as f:
                for item in succeed:
                    f.write(item)
                    f.write('\n')
        UserAgent = 'user-agent=' + random.choice(user_agent_list)
        options.add_argument(UserAgent)
        try:
            driver = webdriver.Chrome(chrome_options=options)
            driver.get(url)
            sleep(3)
            Select(driver.find_element_by_id('fieldSlt-0')).select_by_value('责任者')
            driver.find_element_by_id('criteriaIpt-0').send_keys(scholar.strip())
            sleep(0.3)
            driver.find_element_by_id('yearCriteriaFrom').send_keys('2016')
            sleep(0.3)
            driver.find_element_by_id('yearCriteriaTo').send_keys('2019')
            sleep(0.3)
            driver.find_element_by_class_name('submitBtn').click()
            sleep(2)
            try:
                temp = driver.find_element_by_id('chk_1')
            except:
                print(scholar +' 没有专著信息')
                succeed.append(scholar)
                driver.quit()
                continue
            while True:
                results = driver.find_element_by_id('resultitem-box').find_elements_by_class_name('info') #获得每一条记录数
                temp_list = []
                #driver.switch_to_window(driver.window_handles[0])
                sleep(2)
                for item in results:
                    try:
                        title = item.find_element_by_tag_name('h4').find_element_by_tag_name('span').get_attribute('alt').strip()
                    except:
                        title = item.find_element_by_tag_name('h4').text.strip()
                    f_author = item.find_elements_by_tag_name('p')[1].find_element_by_tag_name('em').text.strip()
                    year = item.find_elements_by_tag_name('p')[2].find_element_by_tag_name('span').find_element_by_tag_name('em').text.strip()
                    publish = item.find_elements_by_tag_name('p')[2].find_elements_by_tag_name('span')[1].find_element_by_tag_name('em').text.strip()
                    row = {'书名':title,'文献类型':'专著','第一作者':f_author,'出版年份':year,'出版社':publish}
                    temp_list.append(row)
                #判断是否有下一页
                try:
                    driver.find_element_by_link_text('下一页').click()
                    sleep(3)
                except:
                    break
            df = pd.DataFrame(temp_list)
            df.to_excel(dpath+scholar+'-专著信息.xlsx')
            succeed.append(scholar)
            print(scholar+' 采集完毕.')
            driver.quit()
            sleep(2)
        except:
            print(scholar + ' 采集异常.')
            driver.quit()
            continue

def to_download(counts):
    global succeed
    scholars = pd.read_excel('首经贸/师资/首都经济贸易大学-应用经济学-师资.xlsx')
    scholars = [x for x in scholars['姓名']]
    for i in range(counts):
        print('第'+str(i+1)+'次循环')
        get_info(scholars)
        succeed = list(set(succeed))
        with open(dpath+'succeed.txt','w',encoding='utf-8') as f:
            for item in succeed:
                f.write(item)
                f.write('\n')

if __name__ == '__main__':
    counts = 3
    threads = []
    for i in range(3):
        t = threading.Thread(target=to_download,args=(counts,))
        threads.append(t)
    for t in threads:
        t.start()
        sleep(0.5)
    for thread in threads:
        thread.join()

第1次循环
第1次循环
第1次循环
郝宇彪 没有专著信息
方兴 采集完毕.
方兴 采集完毕.
黄春元 采集完毕.
李亚男 采集异常.
李婧 采集异常.
苏志 采集异常.
苏志 采集异常.
郎丽华 采集完毕.
李婧 采集异常.
张萍 采集完毕.
李婧 采集异常.
李智 采集异常.
王军 采集异常.
王军 采集完毕.
陈建先 没有专著信息
王军 采集异常.
刘宏 采集异常.
第2次循环
刘宏 采集异常.
刘宏 采集完毕.
李亚男 采集异常.
李亚男 采集异常.
刘宏 采集异常.
苏志 采集异常.
李智 采集完毕.
李婧 采集异常.
第2次循环
苏志 采集完毕.
李亚男 采集异常.
第3次循环
李婧 采集异常.
李亚男 采集完毕.
李智 采集异常.
第2次循环
李亚男 采集异常.
第3次循环
李婧 采集异常.
第3次循环


In [1]:
succeed = list(set(succeed))
with open(dpath+'succeed.txt','w',encoding='utf-8') as f:
    for item in succeed:
        f.write(item)
        f.write('\n')

NameError: name 'succeed' is not defined

### requests + beautiful